# Summary

Recently (possibly since upgrading to openai 0.18.1?), querying openai causes invalid URL errors. Here are some notes from yesterday's troubleshooting session.

- curl works, python doesn't.
- Not just codex, now nox openai engines work w/ python. Maybe due to updating pip package? Temporarily upped billing limit to try other models.
- Restarted kernel and gpt query works again w/ ada. BUT after I import openai explicitly, that fails too. That must be a clue.
- Tried uninstalling, reinstalling, opened new tmux pane. Still same error.
- Tried deleting 'openai' object and then importing jabberwocky. This does work!?
- If I re-import openai after that, gpt.query still works. But openai.completion while codex does not.
- If I import openai FROM jabberwocky openai_utils, codex query still fails. But gpt.query works. And openai.Completion works w/ engine ada!
- Conclusion: maybe it is codex-specific then?

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [4]:
j_kwargs = {'prompt': 'a', 'max_tokens': 1, 'engine_i': 0}
ada_kwargs = {'prompt': 'a', 'max_tokens': 1, 'engine': 'text-ada-001'}
code_kwargs = {'prompt': 'a', 'max_tokens': 1, 'engine': 'code-davinci-001'}

## Jabberwocky

In [5]:
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend
import jabberwocky.openai_utils as oautils

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [6]:
gpt = GPTBackend()
gpt

GPTBackend <current_name: openai>

In [7]:
res = gpt.query(**j_kwargs)

{'max_tokens': 1, 'engine_i': 0, 'prompt': 'a', 'meta': {'backend_name': 'openai', 'query_func': 'query_gpt3', 'datetime': 'Mon Apr 25 19:42:34 2022'}}


In [8]:
res

([''],
 [{'text': '\n',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length',
   'prompt_index': 0}])

## openai 0.6.2

In [9]:
import openai

In [10]:
openai.version.VERSION

'0.18.1'

In [11]:
with open('/Users/hmamin/.openai', 'r') as f:
    openai.api_key = f.read().strip()

In [12]:
# oautils.openai == openai

In [13]:
res = openai.Completion.create(**code_kwargs)

In [14]:
res

<OpenAIObject text_completion id=cmpl-515RxlKh9094yUQFyFqI4pVMKtjrV at 0x11f6c0b48> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "nt"
    }
  ],
  "created": 1650940961,
  "id": "cmpl-515RxlKh9094yUQFyFqI4pVMKtjrV",
  "model": "code-davinci:001",
  "object": "text_completion"
}

## openai 0.18.1

In [15]:
import openai

In [16]:
openai.version.VERSION

'0.18.1'

In [11]:
with open('/Users/hmamin/.openai', 'r') as f:
    openai.api_key = f.read().strip()

In [12]:
# oautils.openai == openai

In [13]:
res = openai.Completion.create(**code_kwargs)

In [14]:
res

<OpenAIObject text_completion id=cmpl-515ItOmu7H4oHPA2yyDXpfHpteeby at 0x1245a1bf8> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1650940399,
  "id": "cmpl-515ItOmu7H4oHPA2yyDXpfHpteeby",
  "model": "code-davinci:001",
  "object": "text_completion"
}

In [14]:
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend
import jabberwocky.openai_utils as oautils

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [15]:
gpt = GPTBackend()
gpt

GPTBackend <current_name: openai>

In [16]:
res = gpt.query(**j_kwargs)

{'max_tokens': 1, 'engine_i': 0, 'prompt': 'a', 'meta': {'backend_name': 'openai', 'query_func': 'query_gpt3', 'datetime': 'Sun Apr 24 15:42:59 2022'}}


In [17]:
res

([''],
 [{'text': '\n',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length',
   'prompt_index': 0}])

## Takeaways

Could not reproduce error with either old or new version of openai package. Maybe ipython handles things differently from jupyter?

Update: could not reproduce in ipython either. No new openai version has been released in the last few days so it's not like they fixed something. Maybe it was an autoreload thing?

## Stop words in streaming mode

In [146]:
from collections import deque
from transformers import GPT2Tokenizer

In [98]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [100]:
tokenizer_j = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-j-6B')

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [199]:
list(tokenizer.get_vocab().items())[-5:]

[('Ġregress', 50252),
 ('ĠCollider', 50253),
 ('Ġinformants', 50254),
 ('Ġgazed', 50255),
 ('<|endoftext|>', 50256)]

In [119]:
tokenizer.tokenize('Sylvia: Hi.\n\nMe: Hello<END>')

['S',
 'yl',
 'via',
 ':',
 'ĠHi',
 '.',
 'Ċ',
 'Ċ',
 'Me',
 ':',
 'ĠHello',
 '<',
 'END',
 '>']

In [200]:
tokenizer.tokenize('Sylvia: Hi.\n\nMe: Hello<|endoftext|>')

['S',
 'yl',
 'via',
 ':',
 'ĠHi',
 '.',
 'Ċ',
 'Ċ',
 'Me',
 ':',
 'ĠHello',
 '<|endoftext|>']

In [120]:
tokenizer_j.tokenize('Sylvia: Hi.\n\nMe: Hello<END>')

['S',
 'yl',
 'via',
 ':',
 'ĠHi',
 '.',
 'Ċ',
 'Ċ',
 'Me',
 ':',
 'ĠHello',
 '<',
 'END',
 '>']

In [17]:
gpt = GPTBackend()
gpt.backends()

['openai', 'gooseai', 'huggingface', 'hobby', 'banana', 'repeat', 'mock']

In [18]:
gpt.switch('mock')

Switching openai backend to "mock".


In [19]:
for tok, full in gpt.query('This is the last time', stream=True):
    print(tok)
    print(full)
    print(spacer())

{'stream': True, 'prompt': 'This is the last time', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock', 'datetime': 'Mon Apr 25 19:42:57 2022'}}
 a
{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x11f6b3db0> JSON: {
  "text_offset": [
    0
  ],
  "token_logprobs": [
    -1.3818359375
  ],
  "tokens": [
    " a"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    }
  ]
}, 'text': ' a', 'token_index': 0, 'prompt_index': 0}

-------------------------------------------------------------------------------

 bit
{'finish_reason': None, 'index': 0, 'logprobs': <OpenAIObject at 0x11f6b3f68> JSON: {
  "text_offset": [
    2
  ],
  "token_logprobs": [
    -4.75390625
  ],
  "tokens": [
    " bit"
  ],
  "top_logprobs": [
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    }
  ]
}, 'text': ' bit', 'token_index': 1, 'prompt_index': 0}

------------------------------

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:721: UserWarning: strip_output=True is not supported in stream mode. Automatically setting it to False.
  warnings.warn('strip_output=True is not supported in stream '
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:726: UserWarning: Streaming mode does not support manual truncation of stop phrases and your current backend has limited support for truncation.
  'Streaming mode does not support manual truncation of '


In [20]:
conv = oautils.ConversationManager(verbose=False)
conv

In [90]:
gpt.switch('openai')
with conv.converse('Sylvia Plath'):
    res = conv.query('Hi.', engine_i=0,
                     stream=True, max_tokens=25)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.
Switching openai backend to "openai".
{'engine_i': 0, 'temperature': 0.5, 'max_tokens': 25, 'frequency_penalty': 0.1, 'stop': ['\n\nMe:', 'This is a conversation with'], 'stream': True, 'prompt': 'This is a conversation with Sylvia Plath. Sylvia Plath (October 27, 1932 - February 11, 1963) was an American poet, novelist, and short-story writer. She is credited with advancing the genre of confessional poetry and is best known for two of her published collections, The Colossus and Other Poems (1960) and Ariel (1965), as well as The Bell Jar, a semi-autobiographical novel published shortly before her death in 1963.\n\nMe: Hi.\n\nSylvia Plath:', 'meta': {'backend_name': 'openai', 'query_func': 'query_gpt3', 'datetime': 'Mon Apr 25 20:25:41 2022'}}


In [91]:
res2 = list(res)

In [92]:
[row[0] for row in res2]

[' Hi', '.', ' How', ' are', ' you', '?', '']

In [208]:
res = [(' Hi',
  {'text': ' Hi',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 ('.',
  {'text': '.',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 (' How',
  {'text': ' How',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 (' are',
  {'text': ' are',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 (' you',
  {'text': ' you',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 ('?',
  {'text': '?',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 ('',
  {'text': '',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
('<|endoftext|>',
  {'text': '<|endoftext|>',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
(' ',
 {'text': ' ',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
 ('Hey',
 {'text': 'Hey',
   'index': 0,
   'logprobs': None,
   'finish_reason': None,
   'prompt_index': 0}),
  (' there',
 {'text': ' there',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length',
   'prompt_index': 0}),
]

Notes on usage:

- When NOT in stream mode, gpt.query uses `openai_utils.truncate_at_first_stop`. This executes on a single completion (np=1, nc=1) at a time.
- When in stream mode, it uses `utils.stream_response`. This calls `stream_openai_generator` if the query func has param 'stream' and does some custom logic in the same stream_response func otherwise.
- Possible way to utilize new stream func (when done): use it in stream_openai_generator? Will be unnecessary for openai (basically just for gooseai, since other query funcs don't support streaming at all) but shouldn't be harmful, in theory. Then update stream_response logic if necessary (might be easier to just convert to str/list and rstrip/replace, since these aren't really streamed).

In [209]:
len(tokenizer.tokenize('<END>'))

3

In [213]:
# Uses lookahead so we don't return any tokens past stop. Might be tricky to
# use w/ multiple stopwords bc diff lengths.
# TODO: update to work when np > 1 and/or nc > 1.
def stream_with_stop(gen, stop_word='<|endoftext|>', stop_word_n_tokens=1):
    # stop_word_n_tokens: number of tokens stop_word consists of when using 
    # the appropriate gpt tokenizer (I vaguely recall this may differ for
    # gpt-j models).
    full_text = ''
    q = deque()
    for i, (text, full) in enumerate(gen):
        full_text += text
        q.append((text, full))
        if i < stop_word_n_tokens: continue
        if full_text.endswith(stop_word):
            q[0][-1]['finish_reason'] = 'stop'
            yield q.popleft()
            break
        yield q.popleft()        

In [211]:
for tok, full in res:
    print(repr(tok), full['finish_reason'])

' Hi' None
'.' None
' How' None
' are' None
' you' None
'?' None
'' None
'<|endoftext|>' None
' ' None
'Hey' None
' there' length


In [212]:
for tok, full in stream_with_stop(res):
    print(repr(tok), full['finish_reason'])

' Hi' None
'.' None
' How' None
' are' None
' you' None
'?' None
'' stop


## Cost estimator

Since the stopword removal for a backend with native streaming currently only helps gooseai, I want to get a better sense of how often gooseai is actually cost effective. Write function to compute:
- openai cost
- gooseai cost
- which one is cheaper
- evaluate over a range of possible prompt lengths and output lengths
    - maybe plot
- consideration: we often specify max_tokens but the actual response could be shorter. Maybe provide an option to use some sort of expected value of return a distribution of possible prices, rather than a single price?

{'gpt-neo-20b': None,
 'gpt-j-6b': None,
 'gpt-neo-2-7b': None,
 'gpt-neo-1-3b': None,
 'gpt-neo-125m': None,
 'fairseq-13b': None,
 'fairseq-6-7b': None,
 'fairseq-2-7b': None,
 'fairseq-1-3b': None,
 'fairseq-125m': None}

In [257]:
def estimate_cost(completion_length, prompt_length=None, prompt=None):
    xor_none(prompt_length, prompt)
    
    # Gooseai base prices (in cents) cover the input and the first 25 tokens 
    # of the output. `Per` prices are cents per token.
    gooseai_prices = {
        'gpt-neo-20b': {'base': 0.002650, 'per': 0.000063},
        'fairseq-13b': {'base': 0.001250, 'per': 0.000036},
        'fairseq-6-7b': {'base': 0.000450, 'per': 0.000012},
        'gpt-j-6b': {'base': 0.000450, 'per': 0.000012},
        'gpt-neo-2-7b': {'base': 0.000300, 'per': 0.000008},
        'fairseq-2-7b': {'base': 0.000300, 'per': 0.000008},
        'gpt-neo-1-3b': {'base': 0.000110, 'per': 0.000003},
        'fairseq-1-3b': {'base': 0.000110, 'per': 0.000003},
        'gpt-neo-125m': {'base': 0.000035, 'per': 0.000001},
        'fairseq-125m': {'base': 0.000035, 'per': 0.000001},
    }
    # Unlike gooseai, openai charges for both prompt and generation tokens.
    # We convert their prices to cents per token.
    # TODO: allow passing in name like 'text-ada-001'?
    # And/or support passing in engine_i int?
    openai_prices = {
        'ada': {'per': .0008 / 1_000},
        'babbage': {'per': .0012 / 1_000},
        'curie': {'per': .0060 / 1_000},
        'davinci': {'per': .0600 / 1_000}
    }
    prompt_length = prompt_length or len(tokenizer.tokenize(prompt))
    gooseai_resolved = [
        ('gooseai', name, prices['base']*prompt_length 
              + prices['per']*max(0, completion_length - 25))
        for name, prices in gooseai_prices.items()
    ]
    openai_resolved = [
        ('openai', name, prices['per'] * (prompt_length + completion_length))
        for name, prices in openai_prices.items()
    ]
    # Prices are returned in cents.
    return pd.DataFrame(
        gooseai_resolved + openai_resolved,
        columns=['backend', 'engine', 'cost_cents']
    ).sort_values('cost_cents', ascending=True).reset_index(drop=True)

In [258]:
prompt = load_prompt('debate', 'A hot dog is a sandwich.')['prompt']
cost_res = estimate_cost(completion_length=300, prompt=prompt)
cost_res

debate: Non-zero frequency penalty was initially included by accident, but in at least 1 test removing it noticeably worsened results.
-------------------------------------------------------------------------------



,backend,engine,cost_cents
0,openai,ada,0.000707
1,openai,babbage,0.001061
2,openai,curie,0.005304
3,gooseai,gpt-neo-125m,0.020715
4,gooseai,fairseq-125m,0.020715
5,openai,davinci,0.053040
6,gooseai,gpt-neo-1-3b,0.065065
7,gooseai,fairseq-1-3b,0.065065
8,gooseai,gpt-neo-2-7b,0.177400
9,gooseai,fairseq-2-7b,0.177400


In [260]:
prompt = conv.kwargs('Jeremy Howard', return_prompt=True)['prompt'] \
    + '\n\nMe: Hi Jeremy.\n\nJeremy Howard:'
estimate_cost(prompt=prompt, completion_length=75)

,backend,engine,cost_cents
0,openai,ada,0.000111
1,openai,babbage,0.000167
2,openai,curie,0.000834
3,gooseai,gpt-neo-125m,0.002290
4,gooseai,fairseq-125m,0.002290
5,gooseai,gpt-neo-1-3b,0.007190
6,gooseai,fairseq-1-3b,0.007190
7,openai,davinci,0.008340
8,gooseai,gpt-neo-2-7b,0.019600
9,gooseai,fairseq-2-7b,0.019600


In [273]:
prompt_lengths = np.arange(1, 2048, 10)
completion_lengths = np.arange(1, 2048, 10)

In [278]:
cost_dfs = {
    (p_len, c_len): estimate_cost(c_len, prompt_length=p_len)
    for p_len, c_len in 
    tqdm(product(prompt_lengths, completion_lengths), 
         total=len(prompt_lengths) * len(completion_lengths))
    if p_len + c_len <= 2_048
}

  0%|          | 0/42025 [00:00<?, ?it/s]

In [279]:
len(cost_dfs)

21115

In [280]:
cost_dfs[1, 1]

,backend,engine,cost_cents
0,openai,ada,0.000002
1,openai,babbage,0.000002
2,openai,curie,0.000012
3,gooseai,gpt-neo-125m,0.000035
4,gooseai,fairseq-125m,0.000035
5,gooseai,gpt-neo-1-3b,0.000110
6,gooseai,fairseq-1-3b,0.000110
7,openai,davinci,0.000120
8,gooseai,gpt-neo-2-7b,0.000300
9,gooseai,fairseq-2-7b,0.000300
